#  GPU CUDA Programming



In [ ]:
!nvidia-smi

Sun Mar 17 19:28:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

The GPU compiler for c++ from Nvidia is called `nvcc`, and is already installed on Colab.

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


## Vector Add




The example file `vector_add.cu` implements the vector addition using 64K threads with CUDA programming.

In [ ]:
%%writefile vector_add.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define size 65536

__global__ void vector_add(int *A, int *B, int *C) {
    int my_id = blockIdx.x * blockDim.x + threadIdx.x;
    C[my_id] = A[my_id] + B[my_id];
}

int main() {
    int i;
    int *A = (int *)malloc(sizeof(int) * size);
    int *B = (int *)malloc(sizeof(int) * size);
    int *C = (int *)malloc(sizeof(int) * size);

    for (i = 0; i < size; i++) {
        A[i] = 1;
        B[i] = 2;
    }

    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int) * size);
    cudaMalloc((void **)&gpu_B, sizeof(int) * size);
    cudaMalloc((void **)&gpu_C, sizeof(int) * size);

    struct timespec start, stop;
    double time;

    cudaMemcpy(gpu_A, A, sizeof(int) * size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int) * size, cudaMemcpyHostToDevice);

    dim3 dimGrid(64);
    dim3 dimBlock(1024);

    if (clock_gettime(CLOCK_REALTIME, &start) == -1) {
        perror("clock gettime");
    }
    vector_add<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C);
    cudaMemcpy(C, gpu_C, sizeof(int) * size, cudaMemcpyDeviceToHost);

    if (clock_gettime(CLOCK_REALTIME, &stop) == -1) {
        perror("clock gettime");
    }
    time = (stop.tv_sec - start.tv_sec) + (double)(stop.tv_nsec - start.tv_nsec) / 1e9;

    printf("Execution time = %f sec\n", time);

    for (i = 0; i < 10; i++) {
        printf("C[%d]=%d ", i, C[i]);
    }

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Writing vector_add.cu


In [ ]:
!nvcc vector_add.cu -o vector_add.out && ./vector_add.out

Execution time = 0.138167 sec
C[0]=3 C[1]=3 C[2]=3 C[3]=3 C[4]=3 C[5]=3 C[6]=3 C[7]=3 C[8]=3 C[9]=3 

## Matrix Multiplacation



In [ ]:
%%writefile mat_mul.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define n 1024

// TODO: Write GPU kernel to perform matrix multiplication
__global__ void matrix_multiply(int *A, int *B, int *C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < n && col < n) {
        int sum = 0;
        for (int i = 0; i < n; ++i) {
            sum += A[row * n + i] * B[i * n + col];
        }
        C[row * n + col] = sum;
    }
}

int main() {
    int i, j;
    int *A = (int *) malloc(sizeof(int)*n*n);
    int *B = (int *) malloc(sizeof(int)*n*n);
    int *C = (int *) malloc(sizeof(int)*n*n);

    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i*n + j] = i;
            B[i*n + j] = i + j;
            C[i*n + j] = 0;
        }
    }

    // TODO: Allocate device memory, use variable names: gpu_A, gpu_B and gpu_C
    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_B, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_C, sizeof(int) * n * n);



    // TODO: Transfer data to device
    cudaMemcpy(gpu_A, A, sizeof(int) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int) * n * n, cudaMemcpyHostToDevice);

    // TODO: Set grid and block sizes
    dim3 dimGrid(32, 32); // Assuming 32x32 threads per block
    dim3 dimBlock(32, 32);

    struct timespec start, stop;
    double time;

    if( clock_gettime( CLOCK_REALTIME, &start) == -1 ) { perror( "clock gettime" );}

    // TODO: Launch kernel
    matrix_multiply<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C);


    // TODO: Transfer data back to host
    cudaMemcpy(C, gpu_C, sizeof(int) * n * n, cudaMemcpyDeviceToHost);

    if( clock_gettime( CLOCK_REALTIME, &stop) == -1 ) { perror( "clock gettime" );}
    time = (stop.tv_sec - start.tv_sec)+ (double)(stop.tv_nsec - start.tv_nsec)/1e9;

    // Print results
    printf("Number of FLOPs = %llu, Execution time = %f sec,\n%lf MFLOPs per sec\n",
        2ULL * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);

    printf("C[100][100]=%d\n", C[100*n + 100]);

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Overwriting mat_mul.cu


In [ ]:
!nvcc mat_mul.cu -o mat_mul.out && ./mat_mul.out

Number of FLOPs = 2147483648, Execution time = 0.119264 sec,
18006.149941 MFLOPs per sec
C[100][100]=62617600


# Blocked Matrix Multiplication


In [ ]:
%%writefile blocked_mat_mul.cu
#include <stdlib.h>
#include <stdio.h>
#include <time.h>

#define n 1024
#define block_size 32

// TODO: Write GPU kernel to perform matrix multiplication
__global__ void blocked_matrix_multiply(int *A, int *B, int *C) {
    __shared__ int shared_A[block_size][block_size];
    __shared__ int shared_B[block_size][block_size];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int row = by * block_size + ty;
    int col = bx * block_size + tx;

    int sum = 0;
    for (int m = 0; m < n / block_size; ++m) {
        shared_A[ty][tx] = A[row * n + m * block_size + tx];
        shared_B[ty][tx] = B[(m * block_size + ty) * n + col];
        __syncthreads();

        for (int k = 0; k < block_size; ++k) {
            sum += shared_A[ty][k] * shared_B[k][tx];
        }
        __syncthreads();
    }
    C[row * n + col] = sum;
}

int main() {
    int i, j;
    int *A = (int *) malloc(sizeof(int)*n*n);
    int *B = (int *) malloc(sizeof(int)*n*n);
    int *C = (int *) malloc(sizeof(int)*n*n);

    for (i = 0; i < n; i++) {
        for (j = 0; j < n; j++) {
            A[i*n + j] = i;
            B[i*n + j] = i + j;
            C[i*n + j] = 0;
        }
    }

    // TODO: Allocate device memory, use variable names: gpu_A, gpu_B and gpu_C
    int *gpu_A, *gpu_B, *gpu_C;
    cudaMalloc((void **)&gpu_A, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_B, sizeof(int) * n * n);
    cudaMalloc((void **)&gpu_C, sizeof(int) * n * n);

    // TODO: Transfer data to device
    cudaMemcpy(gpu_A, A, sizeof(int) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_B, B, sizeof(int) * n * n, cudaMemcpyHostToDevice);


    // TODO: Set grid and block sizes
    dim3 dimGrid(n / block_size, n / block_size);
    dim3 dimBlock(block_size, block_size);


    struct timespec start, stop;
    double time;

    if( clock_gettime( CLOCK_REALTIME, &start) == -1 ) { perror( "clock gettime" );}

    // TODO: Launch kernel
    blocked_matrix_multiply<<<dimGrid, dimBlock>>>(gpu_A, gpu_B, gpu_C);

    // TODO: Transfer data back to host
    cudaMemcpy(C, gpu_C, sizeof(int) * n * n, cudaMemcpyDeviceToHost);

    if( clock_gettime( CLOCK_REALTIME, &stop) == -1 ) { perror( "clock gettime" );}
    time = (stop.tv_sec - start.tv_sec)+ (double)(stop.tv_nsec - start.tv_nsec)/1e9;

    // Print results
    printf("Number of FLOPs = %llu, Execution time = %f sec,\n%lf MFLOPs per sec\n",
        2ULL * n * n * n, time, 1 / time / 1e6 * 2 * n * n * n);

    printf("C[100][100]=%d\n", C[100*n + 100]);

    free(A);
    free(B);
    free(C);
    cudaFree(gpu_A);
    cudaFree(gpu_B);
    cudaFree(gpu_C);
    return 0;
}

Overwriting blocked_mat_mul.cu


In [ ]:
!nvcc blocked_mat_mul.cu -o blocked_mat_mul.out && ./blocked_mat_mul.out

Number of FLOPs = 2147483648, Execution time = 0.063321 sec,
33914.481382 MFLOPs per sec
C[100][100]=62617600
